# Kwame's Zillow Zestimates Error Control

Table of contents with header links goes here.

---

---

## Set up the environment

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# import necessary packages/modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, TweedieRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import IsolationForest

from wrangle import get_zillow_data, prepare_zillow
from preprocessing import zillow_main_split, zillow_Xy_split, impute_nulls, zillow_scale, isolation_forest, concat_dfs, my_RFE
from explore import viz_logerror, corr_heatmap, ttest_viz, ttest_hypo, make_is_1960s, map_1960s, error_heatmap, bath_plot, cluster_log_plot, prop_val_log_plot, county_log_plot, map_k
from model import create_cluster_area, cluster_area_viz, choose_k, intertia_k, cluster_area_dummies, model_1, model_2, model_3, model_1_test

# default viz size settings
plt.rc('figure', figsize=(9, 7))
plt.rc('font', size=13)

# default pandas decimal number display format
#pd.options.display.float_format = '{:20,.2f}'.format

ImportError: cannot import name 'model_1_test' from 'model' (/Users/a666/codeup-data-science/kwames-zillow-zestimates-error-control/model.py)

****
---

## Acquire

In [ ]:
# acquire the zillow data
df = get_zillow_data()
df.shape

---
****

## Prepare

### Tidy the data

In [ ]:
df = prepare_zillow(df)
df.head(3)

### Summarize the clean data

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

The remaining nulls have to be imputed after the data split so that we aren't cheating with our data sets.

### Split the data into train, validate, test.

In [ ]:
# main split
train, validate, test = zillow_main_split(df)

In [ ]:
print(f'Shape of train data: {train.shape}')
print(f'Shape of validate data: {validate.shape}')
print(f'Shape of test data: {test.shape}')

### Impute the remaining nulls with medians.

In [ ]:
train = impute_nulls(train)
train.isnull().sum()

In [ ]:
validate = impute_nulls(validate)
validate.isnull().sum()

In [ ]:
test = impute_nulls(test)
test.isnull().sum()

### Handle outliers

In [ ]:
viz_logerror(train)

**There are outliers present on both ends.**

**Now that I'm on my second iteration through the data science pipeline, I will handle these outliers using an Isolation Forest that detects anomalies.**

In [ ]:
print('Current shape of train:', train.shape)

**Isolation Forest, or iForest for short, is a tree-based anomaly detection algorithm.**

In [ ]:
# temporarily split data into X and y
X_train, X_validate, X_test, y_train, y_validate, y_test = zillow_Xy_split(train, validate, test)

In [ ]:
# Isolation Forest
X_train, X_validate, X_test, y_train, y_validate, y_test = isolation_forest(X_train, X_validate, X_test, y_train, y_validate, y_test)

In [ ]:
# concat the dfs back together
train = concat_dfs(train, X_train, y_train)
validate = concat_dfs(validate, X_validate, y_validate)
test = concat_dfs(test, X_test, y_test)

In [ ]:
# making sure my iForest didn't drop too much data
train.describe()

**Now I'll visualize the distribution of log error again to check that the outliers were removed successfully.**

In [ ]:
viz_logerror(train)

**Looks good, so let's move on to scaling the data.**

### Scale the data

In [ ]:
scaler, train_scaled, validate_scaled, test_scaled = zillow_scale(train, validate, test)
train_scaled.head(3)

****
---

## Explore

### Explore the data and create visualizations

For those unfamiliar with the data, here is a simple correlation heatmap for a quick glance at the features' relationships.

In [ ]:
corr_heatmap(train_scaled.drop(columns=['logerror']))

**It would be interesting to explore the correlation between year built and the number of bathrooms, but in the interest of time I will come back to that in the future, because right now my target is log error.**

In [ ]:
sns.distplot(train.decade)
plt.title('Distribution of decade')

In [ ]:
error_heatmap(train)

**Counties:** Los Angeles = 0, Orange = 1, Ventura = 2

**Takeaways at this point in the pipeline before I handled outliers:**
* Looks like properties built in the 1800s and early 1900s have slightly more log error, especially in Ventura county. Granted, there are not very many properties that meet those criteria, so let's do a statisical test to find out if it's significant or not.
* There seems to be slightly more log error in Orange county.

**Takeaways after I handled outliers:**
* After I handled the outliers, it got rid of all the properties built before 1907.
* Now I see that there is actually less log error in the early 1900s, I am adding a new hypothesis test regarding year built and getting rid of the old one ("There is a difference in Zestimate log error in properties built in the 1800s and the overall log error"), which is no longer applicable.
* I also noticed that my Isolation Forest dropped all Ventura county properties.

In [ ]:
train.century.value_counts(dropna=False)

**My iForest also dropped the 2,000 or so properties that were built in the 2000s.**

****
---

### 1-sample, 2-tailed T-test:

## Is there a significant difference in the log error of Zestimates on properties built in the 1960s and the overall log error?

$
\begin{align*}
   H_0 & : \text{There is no difference in Zestimate log error in properties built in the 1960s and the overall log error.}
   \\
   H_a & : \text{There is a difference in Zestimate log error in properties built in the 1960s and the overall log error.}
   \\
    \alpha & : \text{0.05}
\end{align*}
$

In [ ]:
train.decade.value_counts()

In [ ]:
ttest_viz(train)

I will say that the mean and median are close enough for this iteration, but I could try to improve this in future iterations.

**Now that we know the variable has a normal distribution and we compared the mean and median, we can run the T-test.**

In [ ]:
ttest_hypo(train)

Takeaways from first iteration t-test:

**There doesn't seem to be a statistically significant difference in log error in properties built in the 1800s and the overall log error.**

**However I do notice that the distribution of the log error of properties built in the 1800s is skewed differently (left) from the overall log error.**

Takeaways from second iteration t-test:

**It looks like there IS a significant difference in log error in properties built in the 1960s (versus the overall mean)! I'll make this into a feature.**

## Feature Engineering with Explore findings

Time to add the new feature is_1960s to the data.

In [ ]:
train = make_is_1960s(train)
validate = make_is_1960s(validate)
test = make_is_1960s(test)
train.head(2)

In [ ]:
train_scaled = make_is_1960s(train_scaled)
validate_scaled = make_is_1960s(validate_scaled)
test_scaled = make_is_1960s(test_scaled)
train_scaled.head(2)

In [ ]:
map_1960s(train)

---
---

## Further data exploration

In [ ]:
bath_plot(train)

Less log error on properties with half baths.

In [ ]:
prop_val_log_plot(train)

It seems like there is definitely more error amongst lower value properties.

In [ ]:
county_log_plot(train)

More log error in Los Angeles county.

---
---

## Feature Engineering with Clustering

In [ ]:
train_scaled.columns

In [ ]:
X = train_scaled[['latitude', 'longitude', 'county']]
Xv = validate_scaled[['latitude', 'longitude', 'county']]
Xt = test_scaled[['latitude', 'longitude', 'county']]
kmeans, centroids = create_cluster_area(train, train_scaled, validate, validate_scaled, test, test_scaled, X, Xv, Xt, 5)

In [ ]:
train_scaled.groupby('cluster_area').mean()

In [ ]:
train_scaled.groupby('cluster_area').median()

In [ ]:
sns.pairplot(data=train_scaled.drop(columns=['latitude', 'longitude', 'century', 'decade', 'bathcnt', 'logerror']), hue='cluster_area')

Pipeline iteration 2 Takeaway: **These clusters don't look super useful in distinguishing groups of like properties as of now.**

Pipeline iteration 3 Takeaway: **These clusters could be useful to seperate properties with similar values and year built.**

In [ ]:
intertia_k(X)

**Judging by the elbow method, the sweet spot for my k-value should be around 4 or 5. I'll look closer with a visualization.**

In [ ]:
map_k(X, train)

**I actually really like the region clusters for ```k = 6```, so I'm going to use ```k = 6``` to re-create my new feature (cluster_area) to use in modeling.**

In [ ]:
train = train.drop(columns=['cluster_area'])
train_scaled = train_scaled.drop(columns=['cluster_area'])

In [ ]:
X = train_scaled[['latitude', 'longitude', 'county']]
Xv = validate_scaled[['latitude', 'longitude', 'county']]
Xt = test_scaled[['latitude', 'longitude', 'county']]
kmeans, centroids = create_cluster_area(train, train_scaled, validate, validate_scaled, test, test_scaled, X, Xv, Xt, 6)

In [ ]:
# Now I'll run pairplot on the re-created cluster feature
sns.pairplot(data=train_scaled.drop(columns=['latitude', 'longitude', 'century', 'decade', 'bathcnt', 'logerror', 'is_1960s']), hue='cluster_area')

Pipeline iteration 2 takeaway: **Looks like cluster 0, which only has Los Angeles county properties, has the highest values of all the area clusters in: property value, bathbed count, and square feet. I'll visualize that area in a future pipeline iteration.**

Pipeline iteration 3 takeaway: **There is still a lot of overlap, but it looks like these clusters have somewhat useful distinctions in property values, year built, square feet, and a little in bathbedcnt.**

In [ ]:
cluster_log_plot(train)

Focus on reducing log error in clusters with a bigger range in log error.

**Time to get dummies for the area cluster to make it better to model with.**

In [ ]:
train = cluster_area_dummies(train)
validate = cluster_area_dummies(validate)
test = cluster_area_dummies(test)

In [ ]:
train_scaled = cluster_area_dummies(train_scaled)
validate_scaled = cluster_area_dummies(validate_scaled)
test_scaled = cluster_area_dummies(test_scaled)

In [ ]:
# quick check to make sure my area cluster dummies function is still working
validate_scaled.head(2)

---
---

# Predictive Modeling

**The goal is to produce a predictive model that outperforms the baseline in predicting the target value -- in this case, the log error in the Zestimate of a single-unit property from 2017.**

### Choose Features to use in Modeling with RFE (Recursive Feature Elimination)

In [ ]:
# RFE feature reduction
X = train_scaled.drop(columns=['logerror'])
k = 8

my_RFE(X, k, train_scaled)

**Pipeline iteration 1:**

Based on my RFE results, I will use bathbedcnt, century and cluster_area as predicter features.

I won't use bathcnt or county because the feature redundancy from overlap could overfit my model by effectively giving more weight to some features.

**Pipeline iteration 2:**

Based on my RFE results, I will use bathcnt, sqft, latitude, and yearbuilt as predicter features.

**Pipeline iteration 3:**

Based on my RFE results, I will use (in order of importance): value, yearbuilt, bathbedcnt, cluster_area_5, and cluster_area_1 as predicter features.

I was going to use sqft, but it has a strong correlation with bathbedcnt, so it would be redundant.

The same goes for why I'm dropping latitude in favor of cluster_area_1 and cluster_area_5, and decade for yearbuilt.

## Define and Evaluate Baseline

In [ ]:
np.mean(train.logerror)
#np.median(train.logerror)

Pipeline iteration 1 & 2: **I'm going to go with the median as a baseline prediction for log error since I still have some outliers present in my data.**

Pipeline iteration 3: **I'm going to go with the mean because, according to calculations, it makes a better performing baseline.**

In [ ]:
baseline = train.logerror.mean()

baseline_rmse_train = round(mean_squared_error(train.logerror, np.full(len(train.logerror), baseline))**1/2, 6)
print('RMSE (Root Mean Square Error) of Baseline on train data:\n', baseline_rmse_train)
baseline_rmse_validate = round(mean_squared_error(validate.logerror, np.full(len(validate.logerror), baseline))**1/2, 6)
print('RMSE (Root Mean Square Error) of Baseline on validate data:\n', baseline_rmse_validate)

## Make Models, Evaluate Models, and Test Models

In [ ]:
train_scaled.columns

### Model 1 - Ordinary Least Squares (OLS) using Linear Regression

In [ ]:
# value, yearbuilt, bathbedcnt, cluster_area_5, and cluster_area_1

X = train_scaled.drop(columns=['logerror', 'bathcnt', 'sqft', 'latitude', 'longitude', 'county', 'decade', 'century', 'is_1960s', 'cluster_area_2', 'cluster_area_3', 'cluster_area_4'])
y = train_scaled.logerror

X_v = validate_scaled.drop(columns=['logerror', 'bathcnt', 'sqft', 'latitude', 'longitude', 'county', 'decade', 'century', 'is_1960s', 'cluster_area_2', 'cluster_area_3', 'cluster_area_4'])
y_v = validate_scaled.logerror

lm_pred, lm_rmse, lm_pred_v, lm_rmse_v = model_1(X, y, X_v, y_v)

**This model performs better than the baseline. Yay!**

### Model 2 - Lasso & Lars

In [ ]:
X = train_scaled.drop(columns=['logerror'])
y = train_scaled.logerror

X_v = validate_scaled.drop(columns=['logerror'])
y_v = validate_scaled.logerror

pred_lars, rmse_train, pred_lars_v, rmse_validate = model_2(X, y, X_v, y_v)

**This model performs better than baseline, and almost the same as Model 1, but very slightly less accurate.**

### Model 3

In [ ]:
# try using all features with OLS

X = train_scaled.drop(columns=['logerror'])
y = train_scaled.logerror

X_v = validate_scaled.drop(columns=['logerror'])
y_v = validate_scaled.logerror

lm_pred, lm_rmse, lm_pred_v, lm_rmse_v = model_3(X, y, X_v, y_v)

**This model is obviously overfit, probably because some features are redundant.**

**Now that I know Model 1 is the best performing, I will test it on the test data.**

In [ ]:
# value, yearbuilt, bathbedcnt, cluster_area_5, and cluster_area_1

X = train_scaled.drop(columns=['logerror', 'bathcnt', 'sqft', 'latitude', 'longitude', 'county', 'decade', 'century', 'is_1960s', 'cluster_area_2', 'cluster_area_3', 'cluster_area_4'])
y = train_scaled.logerror

X_v = validate_scaled.drop(columns=['logerror', 'bathcnt', 'sqft', 'latitude', 'longitude', 'county', 'decade', 'century', 'is_1960s', 'cluster_area_2', 'cluster_area_3', 'cluster_area_4'])
y_v = validate_scaled.logerror

lm_pred, lm_rmse = model_1_test(X, y)

---
---

# Conclusion and Takeaways - How to prevent future error in Zestimates